<a href="https://colab.research.google.com/github/LuisEduardoRB/EDP-II/blob/main/M%C3%A9todo_de_Jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Método de Jacobi en Python
# Hecho de forma sencilla, sin librerías especiales.

def jacobi(A, b, x_inicial, tol, max_iter):
    n = len(A)
    x = x_inicial[:]   # copia del vector inicial

    for k in range(max_iter):
        nuevo_x = [0.0] * n
        for i in range(n):
            suma = 0.0
            for j in range(n):
                if j != i:
                    suma += A[i][j] * x[j]
            nuevo_x[i] = (b[i] - suma) / A[i][i]

        # Calcular el error como máximo cambio en los valores
        error = max(abs(nuevo_x[i] - x[i]) for i in range(n))

        # Mostrar cómo va (puedes comentar esto si no quieres tanta salida)
        print("Iteración", k+1, ": x =", [round(val, 6) for val in nuevo_x], "error =", round(error, 8))

        if error < tol:
            break

        x = nuevo_x

    return nuevo_x

# Ejemplo
if __name__ == "__main__":
    # Sistema de ecuaciones:
    # 10x1 - x2 + 2x3 = 6
    # -x1 + 11x2 - x3 = 25
    # 2x1 - x2 + 10x3 = -11

    A = [
        [10, -1,  2],
        [-1, 11, -1],
        [ 2, -1, 10]
    ]
    b = [6, 25, -11]

    # Vector inicial
    x0 = [0, 0, 0]

    resultado = jacobi(A, b, x0, tol=1e-6, max_iter=50)
    print("\nSolución aproximada:", [round(val, 6) for val in resultado])


Iteración 1 : x = [0.6, 2.272727, -1.1] error = 2.27272727
Iteración 2 : x = [1.047273, 2.227273, -0.992727] error = 0.44727273
Iteración 3 : x = [1.021273, 2.277686, -1.086727] error = 0.094
Iteración 4 : x = [1.045114, 2.266777, -1.076486] error = 0.02384132
Iteración 5 : x = [1.041975, 2.269875, -1.082345] error = 0.00585917
Iteración 6 : x = [1.043457, 2.269057, -1.081407] error = 0.00148168
Iteración 7 : x = [1.043187, 2.269277, -1.081786] error = 0.00037814
Iteración 8 : x = [1.043285, 2.269218, -1.08171] error = 9.762e-05
Iteración 9 : x = [1.043264, 2.269234, -1.081735] error = 2.541e-05
Iteración 10 : x = [1.04327, 2.26923, -1.081729] error = 6.66e-06
Iteración 11 : x = [1.043269, 2.269231, -1.081731] error = 1.75e-06
Iteración 12 : x = [1.043269, 2.269231, -1.081731] error = 4.6e-07

Solución aproximada: [1.043269, 2.269231, -1.081731]


In [ ]:
# Método de Jacobi "paso a paso" (sin numpy)
# Resuelve A x = b iterando hasta que la diferencia entre iteraciones sea pequeña.

def es_diagonal_dominante(A):
    """
    Revisa si A es diagonalmente dominante por filas.
    No es obligatorio para que Jacobi funcione, pero ayuda a que converja.
    """
    n = len(A)
    for i in range(n):
        suma_no_diag = 0.0
        for j in range(n):
            if j != i:
                suma_no_diag += abs(A[i][j])
        if abs(A[i][i]) < suma_no_diag:
            return False
    return True

def jacobi(A, b, x0=None, tol=1e-6, max_iter=100):
    """
    Implementación sencilla del método de Jacobi.

    Parámetros:
      A: matriz (lista de listas) de tamaño n x n
      b: vector (lista) de tamaño n
      x0: aproximación inicial (si no se da, usa ceros)
      tol: tolerancia para parar (norma infinito del cambio)
      max_iter: máximo de iteraciones

    Retorna:
      x: aproximación a la solución
      k: número de iteraciones realizadas
      err: error final (norma infinito de x^{k} - x^{k-1})
    """
    n = len(A)

    # Comprobaciones básicas (por si acaso)
    if len(b) != n:
        raise ValueError("El tamaño de b no coincide con el de A.")
    for i in range(n):
        if len(A[i]) != n:
            raise ValueError("A no es cuadrada.")
        if A[i][i] == 0:
            raise ZeroDivisionError(f"Elemento diagonal A[{i}][{i}] es cero (no se puede dividir).")

    # Vector inicial
    if x0 is None:
        x_old = [0.0 for _ in range(n)]
    else:
        if len(x0) != n:
            raise ValueError("El tamaño de x0 no coincide con el de A.")
        x_old = [float(x) for x in x0]

    # (Opcional) Aviso si no es diagonalmente dominante
    if not es_diagonal_dominante(A):
        print("Aviso: A no es diagonalmente dominante por filas. Jacobi puede no converger.")
        print("Aun así, intentaré iterar...\n")

    # Iteraciones de Jacobi
    for k in range(1, max_iter + 1):
        x_new = [0.0 for _ in range(n)]
        for i in range(n):
            suma = 0.0
            for j in range(n):
                if j != i:
                    suma += A[i][j] * x_old[j]
            x_new[i] = (b[i] - suma) / A[i][i]

        # Cálculo del error (norma infinito del cambio)
        err = 0.0
        for i in range(n):
            cambio = abs(x_new[i] - x_old[i])
            if cambio > err:
                err = cambio

        # Mostrar el progreso (quítalo si no quieres tanta salida)
        print(f"Iteración {k:3d} | x = {['{:.6f}'.format(xx) for xx in x_new]} | error = {err:.6e}")

        # Criterio de paro
        if err < tol:
            return x_new, k, err

        # Preparar siguiente vuelta
        x_old = x_new

    # Si llegó aquí, no alcanzó la tolerancia
    return x_old, max_iter, err


# -------------------------
# EJEMPLO DE USO
# -------------------------
if __name__ == "__main__":
    # Ejemplo pequeño (puedes cambiarlo por tu sistema)
    # Sistema:
    #  10x1 - 1x2 + 2x3 = 6
    #  -1x1 + 11x2 - 1x3 = 25
    #   2x1 - 1x2 + 10x3 = -11
    A = [
        [10, -1,  2],
        [-1, 11, -1],
        [ 2, -1, 10]
    ]
    b = [6, 25, -11]

    # Aproximación inicial (si no pones nada, usa ceros)
    x0 = [0, 0, 0]

    # Parámetros de parada
    tol = 1e-8
    max_iter = 100

    x_aprox, iters, err_final = jacobi(A, b, x0=x0, tol=tol, max_iter=max_iter)

    print("\nResultado")
    print("---------")
    print("x aproximada:", ["{:.10f}".format(v) for v in x_aprox])
    print("iteraciones :", iters)
    print("error final :", f"{err_final:.3e}")


Iteración   1 | x = ['0.600000', '2.272727', '-1.100000'] | error = 2.272727e+00
Iteración   2 | x = ['1.047273', '2.227273', '-0.992727'] | error = 4.472727e-01
Iteración   3 | x = ['1.021273', '2.277686', '-1.086727'] | error = 9.400000e-02
Iteración   4 | x = ['1.045114', '2.266777', '-1.076486'] | error = 2.384132e-02
Iteración   5 | x = ['1.041975', '2.269875', '-1.082345'] | error = 5.859174e-03
Iteración   6 | x = ['1.043457', '2.269057', '-1.081407'] | error = 1.481677e-03
Iteración   7 | x = ['1.043187', '2.269277', '-1.081786'] | error = 3.781385e-04
Iteración   8 | x = ['1.043285', '2.269218', '-1.081710'] | error = 9.762183e-05
Iteración   9 | x = ['1.043264', '2.269234', '-1.081735'] | error = 2.541052e-05
Iteración  10 | x = ['1.043270', '2.269230', '-1.081729'] | error = 6.659229e-06
Iteración  11 | x = ['1.043269', '2.269231', '-1.081731'] | error = 1.754292e-06
Iteración  12 | x = ['1.043269', '2.269231', '-1.081731'] | error = 4.640225e-07
Iteración  13 | x = ['1.0432

In [ ]:
# Jacobi para Laplace en [0,2]x[0,2] con h=2/3

def Malla(h):
    # nodos de la malla (en x y en y son iguales porque es cuadrada)
    xs=[0,h,2*h,3*h]
    ys=[0,h,2*h,3*h]
    return xs,ys

def Frontera(u,xs,ys):
    # mete las condiciones de frontera en la malla u
    n=len(xs)
    # lado izquierdo y abajo son 0
    for j in range(n):
        u[0][j]=0
    for i in range(n):
        u[i][0]=0
    # lado derecho: u(2,y)=y(2-y)
    for j in range(n):
        y=ys[j]
        u[-1][j]=y*(2-y)
    # lado de arriba: depende de x
    for i in range(n):
        x=xs[i]
        if 0<x<1:
            u[i][-1]=x
        else:
            u[i][-1]=2-x

def JL(h,tol,max):
    xs,ys=Malla(h)
    n=len(xs)
    # inicializo la malla con ceros
    u=[[0 for _ in range(n)] for _ in range(n)]
    # pongo las condiciones de frontera
    Frontera(u,xs,ys)
    # copio la malla para ir guardando lo nuevo
    nuevo=[[u[i][j] for j in range(n)] for i in range(n)]

    # encabezado para ver los valores en forma de tabla
    print(f"{'Iter':>5} {'Error':>12} {'u11':>12} {'u21':>12} {'u12':>12} {'u22':>12}")
    print("-"*75)

    for k in range(1,max+1):
        # aquí aplico la fórmula de Jacobi a los puntos interiores
        for i in range(1,n-1):
            for j in range(1,n-1):
                nuevo[i][j]=0.25*(u[i+1][j]+u[i-1][j]+u[i][j+1]+u[i][j-1])
        # calculo el error como el máximo cambio
        err=0
        for i in range(1,n-1):
            for j in range(1,n-1):
                dif=abs(nuevo[i][j]-u[i][j])
                if dif>err:
                    err=dif
        # actualizo los valores de u
        for i in range(1,n-1):
            for j in range(1,n-1):
                u[i][j]=nuevo[i][j]

        # imprimo una fila de la tabla con la iteración, error y valores interiores
        print(f"{k:5d} {err:12.6f} {u[1][1]:12.6f} {u[2][1]:12.6f} {u[1][2]:12.6f} {u[2][2]:12.6f}")

        # paro si el error ya es menor que la tolerancia
        if err<tol:
            break

    # al final muestro los nodos y la malla completa
    print("\nNodos x:",[round(x,6) for x in xs])
    print("Nodos y:",[round(y,6) for y in ys])
    print("\nMalla u(i,j):")
    for j in range(n-1,-1,-1):
        fila=[round(u[i][j],6) for i in range(n)]
        print("y=",round(ys[j],2),"|",fila)
    return u,xs,ys,k,err

# aplicamos las fórmulas que ya creamos previamente y le metemos los datos
u,xs,ys,iters,err=JL(h=2/3,tol=1e-6,max=2000)
print("\nTerminó en",iters,"iteraciones con error",err)
print("\nValores interiores finales:")
# muestro los 4 valores que eran incógnitas
interior=[(1,1),(2,1),(1,2),(2,2)]
for (i,j) in interior:
    print("u(",xs[i],",",ys[j],")=",round(u[i][j],6))


 Iter        Error          u11          u21          u12          u22
---------------------------------------------------------------------------
    1     0.388889     0.000000     0.222222     0.166667     0.388889
    2     0.097222     0.097222     0.319444     0.263889     0.486111
    3     0.048611     0.145833     0.368056     0.312500     0.534722
    4     0.024306     0.170139     0.392361     0.336806     0.559028
    5     0.012153     0.182292     0.404514     0.348958     0.571181
    6     0.006076     0.188368     0.410590     0.355035     0.577257
    7     0.003038     0.191406     0.413628     0.358073     0.580295
    8     0.001519     0.192925     0.415148     0.359592     0.581814
    9     0.000760     0.193685     0.415907     0.360352     0.582574
   10     0.000380     0.194065     0.416287     0.360731     0.582954
   11     0.000190     0.194255     0.416477     0.360921     0.583143
   12     0.000095     0.194350     0.416572     0.361016     0.583238
 